In [2]:
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
import datetime



In [4]:
# データ生成用モジュール
# 関数だけ集めておくことにする

# まずこんな感じでsoupが与えられる
soup = BeautifulSoup(open("./jannies.html"),"html.parser")


In [5]:

# ユーティリティ
# 全角→半角
def get_hankaku_from_zenkaku(text):
    return text.translate(str.maketrans({chr(0xFF01 + i): chr(0x21 + i) for i in range(94)}))

# 日付str → datetime
def get_formatted_date(datestr):
    return datetime.datetime.strptime(datestr,"%Y/%m/%d")



In [6]:


#データ整形

# ジャニーズ

class JanniesScraper:                
    
    def __init__(self, df, soup):
        self.df = df
        self.soup = soup
    # 公演名
    # str
    def add_name(df):
        kouen_name_list= list(map(lambda x:x.string.strip().replace("\n",""),soup.select(".list-ticket-name>p")))
        df["name"] = name_list 
        return df 

    #公演日時
    # Timestamp YYYY/mm/dd
    def add_date(df):
        p = re.compile(r"<[^>]*?>")
        date_list = list( map(\
                             lambda x:
                                    datetime.datetime.strptime( #str→datetime
                                        "20"+ #元の文字が19/05/17で年数に不備が生じるので、先頭に20をつける
                                        p.sub("",str(x)) #タグ除去
                                        .replace(" ","") #空白除去
                                        .replace("\n", "")[:8],"%Y/%m/%d"),#改行除去、曜日を削除、フォーマット変換
                             soup.select(".list-ticket-date>span")))
        # type(kouen_data_list[0])
        df["date"] = date_list
        return df

    #チケットデータ収集日  timestamp YYYY/mm/dd
    def add_collected_at(df, datestr):
        df["collected_at"] = datetime.datetime.strptime(datestr,"%Y/%m/%d")
        return df

    #公演までの残り日数 int
    def add_until(df):
        df["until"] = (df["date"] -df["collected_at"]).dt.days  #日時差をint変換
        return df
    
    #  ジャニーズダミー 0 1
    def add_isJannies(df):
        df["isJannies"] = 1
        return df 
        
    # 購入可能ダミー　買えるなら1, 買えないなら0# 買えるとき →　.list-ticket-order内部にformがあるとき
    def add_available(df):
        list_ticket_order = soup.select(".list-ticket-order")
        available = []
        for i, order in enumerate(list_ticket_order):
            if(order.find("form")):
                available.append(1)
            else:
                available.append(0)
        df["available"] = available
        return df
    

        # 取引あたり価格  int 
    def add_deal_price_list(df):
        deal_price_list = list(map(lambda x:
                          int(re.findall(r"￥\d+,\d+",
                                     str(x))[0][1:]
                                     .replace(",", "")),
                          soup.select(".list-ticket-price")))
        df["price_per_deal"] = deal_price_list
        return df

    
    #連番数 int
    def add_unit(df):
        price_list = soup.select(".list-ticket-price")
        unit_list = []
        for prc in price_list:
            if "ticket-sellunit" in str(prc):
                unit_list.append(
                    int(prc.select(".ticket-sellunit")[0].get_text()))
            else:
                unit_list.append(1)
        df["unit"] = unit_list
        return df

#   単位あたり価格 intのつもり
    def add_price_per_unit(df):
        df["price_per_unit"] = df["price_per_deal"]/ df["unit"]
        return df

#     取引あたり価格/連番/単位あたり価格 ただのファサード
    def add_price_info_bundle(self,df):
        self.add_deal_price_list(df) \
            .add_unit(df) \
            .add_price_per_unit()
    
    #　チケット転売施工ダミー 01
    # 収集日によって決定　施工後なら1、施工前なら0
    def is_after_constructed(df):
        construct_date = "2019-06-14"
        df["after_constructed"]=   (df["collected_at"] >=construct_date)*1
        return df
    
    def write_in_csv(df, filename):
        df.to_csv(filename,index=0)


In [ ]:
# テスト
# TODO ScraperインスタンスにDFとSOUPと日付を保持するよう改修
df = pd.DataFrame()
